In [1]:
import jieba
import re
import math
import os
import time
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
# encoding=utf-8
#加载目的路径的文件
def dataload(path):
    with open(path,encoding='utf-8') as f:
        review=f.readlines()
    #用正则表达式匹配删除语料中无用的词性标注和符号
    review1=[re.sub(r"[0-9]+|[-]+|[\s]+|[a-z]+|[A-Z]+|[/]+|[\[]|[\]]","",txt) for txt in review]   
    #按照逗号、句号、感叹号等标点符号分出子句，逐子句应用匹配算法
    pattern = r'\.|\,|;|\?|:|"|”|“|!|，|。|、|；|：|‘|’|！|…|——|……|《|》|（|）'
    review2 =[re.split(pattern, txt) for txt in review1]
    review3=sum(review2,[]) 
    review4=[i for i in review3 if(i != '')]
    return review4

In [3]:
#获取指定路径下的所有文件名
filepath = r'D:\NLPhomework\第四次作业\people1998\199801.txt'
review=dataload(filepath)

In [4]:
#去除特殊符号
nreview=[re.sub(r"[０]|[１]|[２]|[３]|[４]|[５]|[６]|[７]|[８]|[９]|[』]|[『]|[Ａ]|[Ｂ]","",r) for r in review]   
nnreview=[r for r in nreview if r !='']

In [5]:
nnreview

['迈向充满希望的新世纪',
 '一九九八年新年讲话',
 '附图片张',
 '中共中央总书记',
 '国家主席江泽民',
 '一九九七年十二月三十一日',
 '月日',
 '中共中央总书记',
 '国家主席江泽民发表年新年讲话',
 '迈向充满希望的新世纪',
 '新华社记者兰红光摄',
 '同胞们',
 '朋友们',
 '女士们',
 '先生们',
 '在年来临之际',
 '我十分高兴地通过中央人民广播电台',
 '中国国际广播电台和中央电视台',
 '向全国各族人民',
 '向香港特别行政区同胞',
 '澳门和台湾同胞',
 '海外侨胞',
 '向世界各国的朋友们',
 '致以诚挚的问候和良好的祝愿',
 '年',
 '是中国发展历史上非常重要的很不平凡的一年',
 '中国人民决心继承邓小平同志的遗志',
 '继续把建设有中国特色社会主义事业推向前进',
 '中国政府顺利恢复对香港行使主权',
 '并按照',
 '一国两制',
 '港人治港',
 '高度自治的方针保持香港的繁荣稳定',
 '中国共产党成功地召开了第十五次全国代表大会',
 '高举邓小平理论伟大旗帜',
 '总结百年历史',
 '展望新的世纪',
 '制定了中国跨世纪发展的行动纲领',
 '在这一年中',
 '中国的改革开放和现代化建设继续向前迈进',
 '国民经济保持了',
 '高增长',
 '低通胀',
 '的良好发展态势',
 '农业生产再次获得好的收成',
 '企业改革继续深化',
 '人民生活进一步改善',
 '对外经济技术合作与交流不断扩大',
 '民主法制建设',
 '精神文明建设和其他各项事业都有新的进展',
 '我们十分关注最近一个时期一些国家和地区发生的金融风波',
 '我们相信通过这些国家和地区的努力以及有关的国际合作',
 '情况会逐步得到缓解',
 '总的来说',
 '中国改革和发展的全局继续保持了稳定',
 '在这一年中',
 '中国的外交工作取得了重要成果',
 '通过高层互访',
 '中国与美国',
 '俄罗斯',
 '法国',
 '日本等大国确定了双方关系未来发展的目标和指导方针',
 '中国与周边国家和广大发展中国家的友好合作进一步加强',
 '中国积极参与亚太经合组织的活动',
 '参加了东盟—中日韩和中国—东盟首脑非正式会晤',
 '这些外交活动',
 '

In [6]:
#List的元素为每个语句，逐字用空格分割
singlereview= [' '.join(list(r)) for r in nnreview]

In [7]:
singlereview

['迈 向 充 满 希 望 的 新 世 纪',
 '一 九 九 八 年 新 年 讲 话',
 '附 图 片 张',
 '中 共 中 央 总 书 记',
 '国 家 主 席 江 泽 民',
 '一 九 九 七 年 十 二 月 三 十 一 日',
 '月 日',
 '中 共 中 央 总 书 记',
 '国 家 主 席 江 泽 民 发 表 年 新 年 讲 话',
 '迈 向 充 满 希 望 的 新 世 纪',
 '新 华 社 记 者 兰 红 光 摄',
 '同 胞 们',
 '朋 友 们',
 '女 士 们',
 '先 生 们',
 '在 年 来 临 之 际',
 '我 十 分 高 兴 地 通 过 中 央 人 民 广 播 电 台',
 '中 国 国 际 广 播 电 台 和 中 央 电 视 台',
 '向 全 国 各 族 人 民',
 '向 香 港 特 别 行 政 区 同 胞',
 '澳 门 和 台 湾 同 胞',
 '海 外 侨 胞',
 '向 世 界 各 国 的 朋 友 们',
 '致 以 诚 挚 的 问 候 和 良 好 的 祝 愿',
 '年',
 '是 中 国 发 展 历 史 上 非 常 重 要 的 很 不 平 凡 的 一 年',
 '中 国 人 民 决 心 继 承 邓 小 平 同 志 的 遗 志',
 '继 续 把 建 设 有 中 国 特 色 社 会 主 义 事 业 推 向 前 进',
 '中 国 政 府 顺 利 恢 复 对 香 港 行 使 主 权',
 '并 按 照',
 '一 国 两 制',
 '港 人 治 港',
 '高 度 自 治 的 方 针 保 持 香 港 的 繁 荣 稳 定',
 '中 国 共 产 党 成 功 地 召 开 了 第 十 五 次 全 国 代 表 大 会',
 '高 举 邓 小 平 理 论 伟 大 旗 帜',
 '总 结 百 年 历 史',
 '展 望 新 的 世 纪',
 '制 定 了 中 国 跨 世 纪 发 展 的 行 动 纲 领',
 '在 这 一 年 中',
 '中 国 的 改 革 开 放 和 现 代 化 建 设 继 续 向 前 迈 进',
 '国 民 经 济 保 持 了',
 '高 增 长',
 '低 通 胀',
 '的 良 好 发 展 态 势',
 '农 业 生 产 再 次 获 得 好 的 收 成',
 '企 业 改 革 继 续 

In [8]:
#用人民日报1998年语料创建2-gram的词袋模型
cv2 = CountVectorizer(ngram_range=(2, 2),analyzer='word',token_pattern='\w')
cv2_fit=cv2.fit_transform(singlereview)
#所有两个字的组合，以及他们在词袋中的索引
gram2voc=cv2.vocabulary_
#所有两个字的组合在语料库中出现的频数
temp2=cv2_fit.sum(axis=0)
freqlist2=sum(temp2.getA().tolist(),[])

In [9]:
#用人民日报1998年语料创建3-gram的词袋模型
cv3 = CountVectorizer(ngram_range=(3, 3),analyzer='word',token_pattern='\w')
cv3_fit=cv3.fit_transform(singlereview)
#所有两个字的组合，以及他们在词袋中的索引
gram3voc=cv3.vocabulary_
#所有两个字的组合在语料库中出现的频数
temp3=cv3_fit.sum(axis=0)
freqlist3=sum(temp3.getA().tolist(),[])

In [17]:
#2-gram输入联想函数，inputwords为用户输入的单个字，返回联想的前num个结果的列表
def gram2(inputwords,num=1):
    word2dict={}
    flag=False
    for g2 in gram2voc:#在所有两个字的组合中查找第一个字与用户输入字相同的组合
        if(g2[0]==inputwords):
            flag=True
            index=gram2voc[g2]
            freq=freqlist2[index]#获取该组合出现的频数，加入结果字典
            word2dict[g2]=freq
    if(flag==False):
        print('很抱歉，受当前语料库大小限制，无法进行联想。')
        return
    sortword2= sorted(word2dict.items(),key=lambda x:x[1],reverse=True)#降序排序
    if(num<=len(sortword2)):
        output=sortword2[0:num]
    else:
        output=sortword2
    words2=[i[0] for i in output]
    words=[i[2] for i in words2]
    result=[inputwords+i for i in words]
    print('2-gram联想结果为：',result)
    return result

In [11]:
#3-gram输入联想函数，inputwords为用户输入的≥2个字，返回联想的前num个结果的列表
def gram3(inputwords,num=1):
    seg=inputwords[-2:]
    nseg=' '.join(list(seg))
    word3dict={}
    flag=False
    for g3 in gram3voc:#在所有两个字的组合中查找第一个字与用户输入字相同的组合
        if(g3[0:3]==nseg):
            flag=True
            index=gram3voc[g3]
            freq=freqlist3[index]#获取该组合出现的频数，加入结果字典
            word3dict[g3]=freq
    if(flag==False):
        finalword=inputwords[-1]
        result1=gram2(finalword,num)
        return result1
    sortword3= sorted(word3dict.items(),key=lambda x:x[1],reverse=True)#降序排序
    if(num<=len(sortword3)):
        output=sortword3[0:num]
    else:
        output=sortword3
    words3=[i[0] for i in output]
    words=[i[4] for i in words3]
    result=[inputwords+i for i in words]
    print('3-gram联想结果为：',result)
    return result

In [12]:
#对用户输入进行分词
#如果用户输入1个词，则用2-gram；≥1个词，则用3-gram   
if __name__=='__main__':         
    s = input('请输入：')              
    num = int(input('请输入联想结果的数量：'))
    if(len(s)==1):
        result=gram2(s,num)
    elif(len(s)>1):
        result=gram3(s,num)
    else:
        print('输入不能为空！')

请输入：我
请输入联想结果的数量：2
2-gram联想结果为： ['我们', '我国']


In [23]:
#实时响应
from tkinter import *
import datetime
import random
global n,name
n=0
import time

w=Tk('输入法')
w.title('输入法')

var = StringVar()
e1 = Entry(w, text=var)
l1 = Label(w, text='推荐： ', font=("Arial", 12))
e1.pack()
l1.pack()
global content
content = ''
def change():
    time.sleep(1)
    global content
    if content != var.get(): #输入框有变化
        content = var.get()
        if (len(content)==1):
            c = gram2(content) #你的推荐函数（c是推荐结果）
            try:
                l1.config(text='推荐：{}'.format(c[0][1])) 
            except:
                l1.config(text='推荐：') #报错就什么都不推荐
                
        if len(content)>1: #当输入框的文字长度大于1
            c = gram3(content) #你的推荐函数（c是推荐结果）
            try:
                l1.config(text='推荐：{}'.format(c[0][2])) 
            except:
                l1.config(text='推荐：') #报错就什么都不推荐
    l1.after(500,change)

In [24]:
change()
w.mainloop()

2-gram联想结果为： ['我们']
3-gram联想结果为： ['我喜欢']
2-gram联想结果为： ['我们']
3-gram联想结果为： ['我看齐']
2-gram联想结果为： ['太岳']
3-gram联想结果为： ['太好了']
